In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, Input
from tensorflow.keras.layers import Layer
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.models import Model
import numpy as np


class H1Layer(Layer):
    def __init__(self, **kwargs):
        super(H1Layer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.b = self.add_weight(shape=(input_shape[-1],),
                                initializer=RandomNormal(mean=0.0,stddev=0.03),
                                trainable=True)
        super(H1Layer, self).build(input_shape)

    def call(self, x):
        return self.b * (2 * x)
        #return (2 * x) 


class H2Layer(Layer):
    def __init__(self, **kwargs):
        super(H2Layer, self).__init__(**kwargs)

    def call(self, x, h1):
        return (2*x*(h1))-2
    
class H3Layer(Layer):
    def __init__(self, **kwargs):
        super(H3Layer, self).__init__(**kwargs)
        
    def call(self, x, h1, h2):
        return (2 * x * (h2))-(4 * h1)

class H4Layer(Layer):
    def __init__(self, **kwargs):
        super(H4Layer, self).__init__(**kwargs)

    def call(self, x, h2, h3):
        return (2*x*(h3))-(6*h2)   
def softsign(x):
    return x / (1 + tf.abs(x))



2024-02-23 12:18:07.237634: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-23 12:18:07.744896: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:

class ResNetHermiteBlock(layers.Layer):
    def __init__(self, filters, stride=1, activation='hermite', **kwargs):
        super(ResNetHermiteBlock, self).__init__(**kwargs)
        self.stride = stride
        self.filters = filters
        self.activation = activation

        # Conditional initialization of Hermite Polynomial layers
        if self.activation == 'hermite':
            self.h1 = H1Layer()
            self.h2 = H2Layer()
            self.h3 = H3Layer()
            self.h4 = H4Layer()

        # Initialize Convolutional and Batch Normalization layers
        self.conv1 = layers.Conv2D(filters, 3, strides=stride, padding="same", use_bias=False)
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(filters, 3, padding="same", use_bias=False)
        self.bn2 = layers.BatchNormalization()

        # Pre-instantiate the Conv2D layer for shortcut adjustment
        self.shortcut_conv = layers.Conv2D(filters, 1, strides=stride, padding="same", use_bias=False)
        self.shortcut_bn = layers.BatchNormalization()

    def call(self, inputs):
        x = inputs

        x = self.conv1(x)
        x = self.bn1(x)

        # Apply Hermite or ReLU based on the activation option
        if self.activation == 'hermite':
            x_h1 = self.h1(x)  # Store H1 output if needed for H2
            x_h2 = self.h2(x,x_h1)  # Store H1 output if needed for H2
            x_h3 = self.h3(x,x_h1,x_h2)  # Store H1 output if needed for H2
            x = self.conv2(x)
            x = self.bn2(x)
            x = self.h4(x,x_h2,x_h3)  # Apply H4 on the convolved output
        else:  # ReLU activation
            x = tf.nn.relu(x)
            x = self.conv2(x)
            x = self.bn2(x)
            x = tf.nn.relu(x)

        # Conditional application for shortcut
        if self.stride != 1 or inputs.shape[-1] != self.filters:
            shortcut = self.shortcut_conv(inputs)
            shortcut = self.shortcut_bn(shortcut)
        else:
            shortcut = inputs

        x += shortcut
        return x


def build_resnet18_hermite(input_shape, num_classes, activation='hermite'):
    inputs = Input(shape=input_shape)
    x = layers.Conv2D(64, 3, strides=1, padding="same", use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    
    # Choose the activation function based on the parameter
    if activation == 'hermite':
        x = H1Layer()(x)  # Hermite activation
    else:
        x = layers.ReLU()(x)  # ReLU activation

    # Define blocks with the activation option
    x = ResNetHermiteBlock(64, stride=1)(x)
    x = ResNetHermiteBlock(64, stride=1)(x)
    x = ResNetHermiteBlock(128, stride=2)(x)
    x = ResNetHermiteBlock(128, stride=1)(x)
    # x = ResNetHermiteBlock(256, stride=2)(x)
    # x = ResNetHermiteBlock(256, stride=1)(x)
    # x = ResNetHermiteBlock(512, stride=2)(x)
    # x = ResNetHermiteBlock(512, stride=1)(x)    # Repeat for other blocks...

    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model


# Example usage
model = build_resnet18_hermite((32, 32, 3), 10, activation='hermite')
# model = build_resnet18_hermite((32, 32, 3), 10, activation='relu')

model.summary()

2024-02-23 12:18:08.417707: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-23 12:18:08.436495: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-23 12:18:08.436678: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 64)        1728      
                                                                 
 batch_normalization (Batch  (None, 32, 32, 64)        256       
 Normalization)                                                  
                                                                 
 h1_layer (H1Layer)          (None, 32, 32, 64)        64        
                                                                 
 res_net_hermite_block (Res  (None, 32, 32, 64)        74304     
 NetHermiteBlock)                                                
                                                                 
 res_net_hermite_block_1 (R  (None, 32, 32, 64)        74304 

In [3]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


# Load CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=0)

print('Train Images Shape:      ', X_train.shape)
print('Train Labels Shape:      ', y_train.shape)

# print('\nValidation Images Shape: ', X_valid.shape)
# print('Validation Labels Shape: ', y_valid.shape)

print('\nTest Images Shape:       ', X_test.shape)
print('Test Labels Shape:       ', y_test.shape)

# CIFAR-10 classes
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
# Convert pixel values data type to float32
X_train = X_train.astype('float32')
X_test  = X_test.astype('float32')
# X_valid = X_valid.astype('float32')
# Calculate the mean and standard deviation of the training images
mean = np.mean(X_train)
std  = np.std(X_train)

# Normalize the data
# The tiny value 1e-7 is added to prevent division by zero
X_train = (X_train-mean)/(std+1e-7)
X_test  = (X_test-mean) /(std+1e-7)
# X_valid = (X_valid-mean)/(std+1e-7)

# Convert class vectors to binary class matrices (one-hot encoding)
y_train = to_categorical(y_train, 10)
# y_valid = to_categorical(y_valid, 10)
y_test  = to_categorical(y_test, 10)

Train Images Shape:       (50000, 32, 32, 3)
Train Labels Shape:       (50000, 1)

Test Images Shape:        (10000, 32, 32, 3)
Test Labels Shape:        (10000, 1)


In [4]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train,
                    batch_size=64,
                    epochs=50,
                    verbose=2,
                    validation_data=(X_test, y_test))


Epoch 1/50


2024-02-23 12:18:13.627630: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2024-02-23 12:18:14.332936: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-02-23 12:18:14.335462: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fdf1c012a50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-23 12:18:14.335492: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2024-02-23 12:18:14.341735: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-02-23 12:18:14.444674: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.

782/782 - 22s - loss: 243.5976 - accuracy: 0.1334 - val_loss: 277848288.0000 - val_accuracy: 0.1008 - 22s/epoch - 28ms/step
Epoch 2/50
782/782 - 16s - loss: 6.2823 - accuracy: 0.1933 - val_loss: 4.9735 - val_accuracy: 0.1559 - 16s/epoch - 20ms/step
Epoch 3/50
782/782 - 16s - loss: 3.8415 - accuracy: 0.2132 - val_loss: 2.9299 - val_accuracy: 0.1875 - 16s/epoch - 20ms/step
Epoch 4/50
782/782 - 16s - loss: 2.4720 - accuracy: 0.2386 - val_loss: 2.5864 - val_accuracy: 0.2159 - 16s/epoch - 20ms/step
Epoch 5/50
782/782 - 16s - loss: 2.2691 - accuracy: 0.2507 - val_loss: 2.4078 - val_accuracy: 0.2528 - 16s/epoch - 20ms/step
Epoch 6/50
782/782 - 16s - loss: 2.1711 - accuracy: 0.2640 - val_loss: 2.6782 - val_accuracy: 0.2308 - 16s/epoch - 20ms/step
Epoch 7/50
782/782 - 16s - loss: 2.8551 - accuracy: 0.2556 - val_loss: 2.3393 - val_accuracy: 0.2772 - 16s/epoch - 20ms/step
Epoch 8/50
782/782 - 16s - loss: 2.1256 - accuracy: 0.2782 - val_loss: 2.1915 - val_accuracy: 0.2841 - 16s/epoch - 20ms/step
E

KeyboardInterrupt: 